## Import Libraries

In [1]:
import pandas as pd
import requests
from io import BytesIO
from collections import defaultdict

## Retrieve Data

In [2]:
# Define URLs for the datasets
restaurant_link = 'https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json'
country_codes_link = 'https://github.com/Papagoat/brain-assessment/blob/main/Country-Code.xlsx?raw=true'

# Fetch restaurant data in JSON format
response_restaurants = requests.get(restaurant_link)
restaurants_json = response_restaurants.json()

# Fetch country codes from Excel
response_country_codes = requests.get(country_codes_link)
country_codes = pd.read_excel(BytesIO(response_country_codes.content))

# Display the country codes data to verify it has been loaded correctly
country_codes

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia
5,148,New Zealand
6,162,Phillipines
7,166,Qatar
8,184,Singapore
9,189,South Africa


## Extract Data

## Question 1: Data Extraction for Restaurants

In [3]:
# Initialize a list to hold restaurant information
restaurants_info_list = []

# Loop through each entry in the restaurant data
for restaurant_group in restaurants_json:
    for restaurant_data in restaurant_group['restaurants']:
        restaurant = restaurant_data['restaurant']

        # Check for associated events
        if 'zomato_events' in restaurant and restaurant['zomato_events']:
            for event_data in restaurant['zomato_events']:
                event_details = event_data.get('event', {})

                if 'photos' in event_details and event_details['photos']:
                    photo_data = event_details['photos'][0].get('photo', {})
                    event_photo_url = photo_data.get('url', 'No Photo Available')
                    thumbnail_url = photo_data.get('thumb_url', 'No Thumbnail Available')
                else:
                    event_photo_url = 'No Photo Available'
                    thumbnail_url = 'No Thumbnail Available'
                
                # Compile restaurant and event information
                restaurant_entry = {
                    'Restaurant_ID': restaurant['R']['res_id'],
                    'Restaurant_Name': restaurant['name'],
                    'Country_ID': restaurant['location']['country_id'],
                    'City_Name': restaurant['location']['city'],
                    'Total_Votes': restaurant['user_rating']['votes'],
                    'Average_Rating': float(restaurant['user_rating']['aggregate_rating']),
                    "Rating_Text": restaurant["user_rating"]["rating_text"],
                    'Cuisine_Types': restaurant['cuisines'],
                    'Event_ID': event_details.get('event_id', "NA"),
                    'Event_Title': event_details.get('title', "NA"),
                    'Event_Start_Date': event_details.get('start_date', "NA"),
                    'Event_End_Date': event_details.get('end_date', "NA"),
                    'Event_Start_Time': event_details.get('start_time', "NA"),
                    'Event_End_Time': event_details.get('end_time', "NA"),
                    'Event_Photo_URL': event_photo_url,
                    'Thumbnail_URL': thumbnail_url
                }

                # Append the compiled data to the list
                restaurants_info_list.append(restaurant_entry)
        else:
            # If no events, include only restaurant information
            restaurant_entry = {
                'Restaurant_ID': restaurant['R']['res_id'],
                'Restaurant_Name': restaurant['name'],
                'Country_ID': restaurant['location']['country_id'],
                'City_Name': restaurant['location']['city'],
                'Total_Votes': restaurant['user_rating']['votes'],
                'Average_Rating': float(restaurant['user_rating']['aggregate_rating']),
                "Rating_Text": restaurant["user_rating"]["rating_text"],
                'Cuisine_Types': restaurant['cuisines'],
                'Event_ID': "NA",
                'Event_Title': "NA",
                'Event_Start_Date': "NA",
                'Event_End_Date': "NA",
                'Event_Start_Time': "NA",
                'Event_End_Time': "NA",
                'Event_Photo_URL': "NA",
                'Thumbnail_URL': "NA"
            }

            restaurants_info_list.append(restaurant_entry)

# Convert the list to a DataFrame
restaurants_dataframe = pd.DataFrame(restaurants_info_list)
restaurants_dataframe

,Restaurant_ID,Restaurant_Name,Country_ID,City_Name,Total_Votes,Average_Rating,Rating_Text,Cuisine_Types,Event_ID,Event_Title,Event_Start_Date,Event_End_Date,Event_Start_Time,Event_End_Time,Event_Photo_URL,Thumbnail_URL
0,18649486,The Drunken Botanist,1,Gurgaon,4765,4.4,Very Good,"Continental, Italian, North Indian, Chinese",322331,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28,20:00:00,23:59:59,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
1,308322,Hauz Khas Social,1,New Delhi,13627,4.6,Excellent,"Continental, American, Asian, North Indian, Ch...",332812,Live 20/20 Match Screenings,2019-03-29,2019-05-23,09:00:00,01:35:00,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
2,18856789,AIR- An Ivory Region,1,New Delhi,1819,4.1,Very Good,"North Indian, Chinese, Continental, Asian",336644,Dhol Bhangra Night,2019-04-10,2019-04-11,21:00:00,00:00:00,https://b.zmtcdn.com/data/zomato_events/photos...,https://b.zmtcdn.com/data/zomato_events/photos...
3,307374,AMA Cafe,1,New Delhi,3252,4.4,Very Good,"Cafe, Juices",NA,NA,NA,NA,NA,NA,NA,NA
4,18238278,Tamasha,1,New Delhi,8112,4.4,Very Good,"Finger Food, North Indian, Continental, Italian",NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,7900186,Mexico,148,Hamilton,100,4.2,Very Good,Mexican,NA,NA,NA,NA,NA,NA,NA,NA
1396,7900182,Thai House Express,148,Hamilton,51,4.0,Very Good,Thai,NA,NA,NA,NA,NA,NA,NA,NA
1397,7900023,Indian Star,148,Hamilton,63,3.8,Good,"North Indian, Indian",NA,NA,NA,NA,NA,NA,NA,NA
1398,7900490,Zealong Tea Estate,148,Hamilton,55,4.7,Excellent,"European, Asian, Desserts",NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
#Merge restaurant information with country codes
merged_restaurants_df = pd.merge(restaurants_dataframe, country_codes, how='left', left_on='Country_ID', right_on='Country Code')

# Select and rename columns, and remove duplicates
final_restaurants_df = merged_restaurants_df[[
    'Restaurant_ID', 
    'Restaurant_Name', 
    'Country', 
    'City_Name', 
    'Total_Votes', 
    'Average_Rating',
    'Cuisine_Types'
]].drop_duplicates()

# Save the cleaned DataFrame to a CSV file
final_restaurants_df.to_csv('restaurants.csv', index=False)